In [12]:
from pyspark.sql import SparkSession
from os import walk
import glob

jars = glob.glob('./jars/*.jar')
print(jars)

!export AWS_ACCESS_KEY_ID=root
!export AWS_SECRET_ACCESS_KEY=12345678

spark = SparkSession.builder\
    .config("spark.jars",",".join(jars))\
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.my_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.my_catalog.catalog-impl", "org.apache.iceberg.jdbc.JdbcCatalog") \
    .config("spark.sql.catalog.my_catalog.uri", "jdbc:mysql://host.docker.internal:3306/iceberg_catalog") \
    .config("spark.sql.catalog.my_catalog.jdbc.user", "root") \
    .config("spark.sql.catalog.my_catalog.jdbc.password", "root") \
    .config("spark.sql.catalog.my_catalog.jdbc.driver", "com.mysql.cj.jdbc.Driver") \
    .config("spark.sql.catalog.my_catalog.warehouse", "s3a://warehouse") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://host.docker.internal:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "root") \
    .config("spark.hadoop.fs.s3a.secret.key", "12345678") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

['./jars/hadoop-aws-3.3.2.jar', './jars/aws-java-sdk-bundle-1.11.1026.jar', './jars/iceberg-spark-runtime-3.5_2.12-1.5.2.jar', './jars/mysql-connector-j-8.4.0.jar']


In [2]:
jdbc_url = "jdbc:mysql://host.docker.internal:3306/spark_test"
table_name = "config"
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# 读取MySQL表数据
df = spark.read.jdbc(url=jdbc_url, table=table_name, properties=properties)

In [3]:
df.show()

+----+----+
| key| val|
+----+----+
|NAME|andy|
+----+----+



In [10]:
spark.sql("""
CREATE TABLE IF NOT EXISTS my_catalog.default.my_iceberg_table (
    key STRING,
    val STRING
) USING iceberg
""")

DataFrame[]

In [11]:
df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .save("my_catalog.default.my_iceberg_table")